In [2]:
pip install torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 MB 26.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: torch
    Found existing installation: torch 2.8.0
    Uninstalling torch-2.8.0:
      Successfully uninstalled torch-2.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1 requires torch==2.2.1, but you have torch 2.9.0 which is incompatible.

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
conda install pytorch torchvision torchaudio cpuonly -c pytorch -c conda-forge

Retrieving notices: ...working... done

CondaValueError: You have chosen a non-default solver backend (libmamba) but it was not recognized. Choose one of: classic


Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install imagehash timm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 13.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### 1. Baseline CNN

In [ ]:
# ====== Minimal Car Damage Pipeline: EDA -> Train -> Report ======
from pathlib import Path
import os, random, json
import numpy as np, pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import timm

from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.preprocessing import label_binarize


TRAIN_DIR = Path("training")  
TEST_DIR  = Path("validation")

assert TRAIN_DIR.is_dir(), f"TRAIN_DIR not found: {TRAIN_DIR}"
assert TEST_DIR.is_dir(),  f"TEST_DIR not found: {TEST_DIR}"

SEED=42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
IMSIZE = 224
IMNET_MEAN = np.array([0.485,0.456,0.406], dtype="float32")
IMNET_STD  = np.array([0.229,0.224,0.225], dtype="float32")
IMG_EXT = {".jpg",".jpeg",".png",".bmp",".tif",".tiff"}
CLASSES = ["minor","moderate","severe"]
LBL = {c:i for i,c in enumerate(CLASSES)}

ART = Path("artifacts"); ART.mkdir(exist_ok=True)
print("Device:", DEVICE)

# ---- 1) Scan folders ----
def label_from_name(name:str):
    n=name.lower()
    if "minor" in n: return "minor"
    if "moderate" in n: return "moderate"
    if "severe" in n: return "severe"
    return None

def scan_split(split_dir: Path, split_name: str):
    rows=[]
    for pdir in sorted([d for d in split_dir.iterdir() if d.is_dir()]):
        lab = label_from_name(pdir.name)
        if lab is None: 
            continue
        for img in pdir.glob("*"):
            if img.suffix.lower() in IMG_EXT:
                rows.append({"path": str(img.resolve()), "label": lab, "split": split_name})
    return rows

df = pd.DataFrame(scan_split(TRAIN_DIR,"train") + scan_split(TEST_DIR,"test"))
assert len(df)>0, "No images found—check paths."
df = df.sample(frac=1.0, random_state=SEED).reset_index(drop=True)

# ---- 2) (Simple) EDA ----
counts = df.groupby(["split","label"]).size().unstack(fill_value=0)
print("\nClass counts by split:\n", counts)
counts.to_csv(ART/"class_counts_by_split.csv")

test_counts = counts.loc["test"]
baseline_label = test_counts.idxmax()
baseline_acc = test_counts.max() / test_counts.sum()
print(f"Test majority baseline: {baseline_label} = {baseline_acc:.3f}")

# ---- 3) Tiny image check: drop unreadable files upfront ----
def is_good_image(p):
    try:
        with Image.open(p) as im:
            im.verify()
        return True
    except Exception:
        return False

mask = df["path"].map(is_good_image)
bad = df.loc[~mask, "path"].tolist()
if bad:
    print(f"Dropping {len(bad)} unreadable images")
df = df[mask].reset_index(drop=True)

# ---- 4) Datasets (no torchvision, no multiprocessing pitfalls) ----
def to_tensor(img: Image.Image):
    img = img.convert("RGB").resize((IMSIZE,IMSIZE))
    arr = np.asarray(img).astype("float32")/255.0
    arr = (arr - IMNET_MEAN) / IMNET_STD
    arr = arr.transpose(2,0,1)  # CHW
    return torch.from_numpy(arr)

class FrameDS(Dataset):
    def __init__(self, frame: pd.DataFrame, train: bool=False):
        self.df = frame.reset_index(drop=True).copy()
        self.train = train
    def __len__(self): return len(self.df)
    def __getitem__(self, idx: int):
        row = self.df.iloc[idx]
        img = Image.open(row["path"])
        x = to_tensor(img)
        y = LBL[row["label"]]
        return x, y

train_df = df[df.split=="train"].copy()
test_df  = df[df.split=="test"].copy()

train_loader = DataLoader(FrameDS(train_df, True), batch_size=32, shuffle=True,  num_workers=0, pin_memory=False)
test_loader  = DataLoader(FrameDS(test_df,  False), batch_size=64, shuffle=False, num_workers=0, pin_memory=False)



Device: cpu

Class counts by split:
 label  minor  moderate  severe
split                         
test      82        75      91
train    452       463     468
Test majority baseline: severe = 0.367


In [4]:
import math

@torch.no_grad()
def compute_nll_and_counts(model, loader):
    """
    Compute total negative log-likelihood and number of samples.
    Uses CrossEntropyLoss with reduction='sum', which matches NLL.
    """
    model.eval()
    total_nll = 0.0
    total_n = 0

    for x, y in loader:
        x, y = x.to(DEVICE), y.to(DEVICE)
        logits = model(x)
        # sum of per-sample NLL
        nll = nn.CrossEntropyLoss(reduction="sum")(logits, y)
        total_nll += nll.item()
        total_n += x.size(0)

    return total_nll, total_n


def compute_aic_bic(model, loader, variant_name: str, out_json_path: Path):
    """
    Compute AIC and BIC for a classification model on a given loader.
    Saves results to JSON under artifacts.
    """
    nll, n = compute_nll_and_counts(model, loader)

    # number of trainable parameters
    k = sum(p.numel() for p in model.parameters() if p.requires_grad)

    aic = 2 * k + 2 * nll
    bic = k * math.log(n) + 2 * nll

    print(f"\n===== {variant_name}: AIC / BIC =====")
    print(f"Samples (n): {n}")
    print(f"Parameters (k): {k}")
    print(f"Negative log-likelihood (NLL): {nll:.2f}")
    print(f"AIC: {aic:.2f}")
    print(f"BIC: {bic:.2f}")

    with open(out_json_path, "w") as f:
        json.dump(
            {
                "variant": variant_name,
                "n": int(n),
                "k": int(k),
                "nll": float(nll),
                "aic": float(aic),
                "bic": float(bic),
            },
            f,
            indent=2,
        )

    return aic, bic


In [ ]:
# ---- 5) Model (Variant: ResNet18 fine-tune via timm) ----
from sklearn.model_selection import StratifiedKFold
from matplotlib import pyplot as plt

def make_model(num_classes=3, pretrained=True):
    """
    ResNet18 fine-tuning model for minimal car damage classification.
    """
    return timm.create_model("resnet18.a1_in1k", pretrained=pretrained, num_classes=num_classes)

# ====== 5.1 Single train/test run (Checkpoint 1 original pipeline) ======

# Dataloaders for train/test split
train_loader = DataLoader(
    FrameDS(train_df, True),
    batch_size=32, shuffle=True, num_workers=0, pin_memory=False
)
test_loader  = DataLoader(
    FrameDS(test_df, False),
    batch_size=64, shuffle=False, num_workers=0, pin_memory=False
)

# Model, loss, optimizer
model = make_model(num_classes=3, pretrained=True).to(DEVICE)

# Class weights based on training set (handle imbalance)
tc = train_df["label"].value_counts().reindex(CLASSES, fill_value=0).astype(float)
N = tc.sum(); K = len(CLASSES)
w = torch.tensor([N / (K * tc[c]) for c in CLASSES], dtype=torch.float32, device=DEVICE)

criterion = nn.CrossEntropyLoss(weight=w)
opt = torch.optim.Adam(model.parameters(), lr=1e-4)
EPOCHS = 8  # you can adjust

def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total = 0.0
    for x, y in loader:
        x = x.to(DEVICE); y = y.to(DEVICE)
        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        total += loss.item() * x.size(0)
    return total / len(loader.dataset)

@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    yT, yP, yPR = [], [], []
    for x, y in loader:
        x = x.to(DEVICE)
        logits = model(x)
        yP.extend(torch.argmax(logits, 1).cpu().numpy())
        yPR.extend(torch.softmax(logits, 1).cpu().numpy())
        yT.extend(y.numpy())
    yT = np.array(yT); yP = np.array(yP); yPR = np.array(yPR)
    acc = accuracy_score(yT, yP)
    f1m = f1_score(yT, yP, average="macro")
    rep = classification_report(yT, yP, target_names=CLASSES, output_dict=True)
    return acc, f1m, rep, yT, yP, yPR

print("===== Train/Test run for Checkpoint-1 model variant =====")
best = 0.0
for ep in range(1, EPOCHS + 1):
    tr_loss = train_one_epoch(model, train_loader, opt, criterion)
    acc, f1m, _, _, _, _ = evaluate(model, test_loader)
    print(f"Epoch {ep:02d} | train_loss={tr_loss:.4f} | test_acc={acc:.3f} | macroF1={f1m:.3f}")
    if acc > best:
        best = acc
        torch.save(model.state_dict(), ART / "best_checkpoint1.pt")

# Final test evaluation
model.load_state_dict(torch.load(ART / "best_checkpoint1.pt", map_location=DEVICE))
acc, f1m, rep, y_true, y_pred, y_prob = evaluate(model, test_loader)
pd.DataFrame(rep).to_csv(ART / "classification_report_checkpoint1.csv")
print(f"\nFINAL — Test Acc: {acc:.3f} | Macro-F1: {f1m:.3f}")

# 1) Baseline vs model accuracy (test set)
plt.figure(figsize=(5, 4))
vals = [baseline_acc, acc]
plt.bar(["Naive baseline", "CNN (ResNet18 FT)"], vals)
plt.ylim(0, 1); plt.ylabel("Accuracy")
for i, v in enumerate(vals):
    plt.text(i, v + 0.02, f"{v*100:.1f}%", ha="center")
plt.title("Accuracy: Baseline vs CNN (Test Set)")
plt.tight_layout()
plt.savefig(ART / "bar_accuracy_checkpoint1.png", dpi=150)
plt.close()

# 2) Confusion matrix
cm = confusion_matrix(y_true, y_pred, labels=[0, 1, 2])
fig, ax = plt.subplots(figsize=(5.2, 4.6))
im = ax.imshow(cm, cmap="Blues")
ax.set_xticks([0, 1, 2]); ax.set_xticklabels([c.capitalize() for c in CLASSES], rotation=15)
ax.set_yticks([0, 1, 2]); ax.set_yticklabels([c.capitalize() for c in CLASSES])
for (i, j), v in np.ndenumerate(cm):
    ax.text(j, i, str(v), ha="center", va="center")
ax.set_xlabel("Predicted"); ax.set_ylabel("True"); ax.set_title("Confusion Matrix (Test)")
fig.colorbar(im, fraction=0.046, pad=0.04)
plt.tight_layout()
plt.savefig(ART / "confusion_matrix_checkpoint1.png", dpi=150)
plt.close()

# 3) ROC (OvR) + macro AUC
y_true_ovr = label_binarize(y_true, classes=[0, 1, 2])
auc_macro = roc_auc_score(y_true_ovr, y_prob, average="macro", multi_class="ovr")
plt.figure(figsize=(5.6, 4.6))
for i, c in enumerate(CLASSES):
    fpr, tpr, _ = roc_curve(y_true_ovr[:, i], y_prob[:, i])
    plt.plot(fpr, tpr, label=c.capitalize())
plt.plot([0, 1], [0, 1], "--", linewidth=1)
plt.xlabel("FPR"); plt.ylabel("TPR")
plt.title(f"ROC (OvR) • Macro-AUC={auc_macro:.3f}")
plt.legend()
plt.tight_layout()
plt.savefig(ART / "roc_ovr_checkpoint1.png", dpi=150)
plt.close()

with open(ART / "metrics_checkpoint1_test.json", "w") as f:
    json.dump(
        {
            "baseline_acc": float(baseline_acc),
            "test_acc": float(acc),
            "macro_f1": float(f1m),
            "macro_auc_ovr": float(auc_macro),
        },
        f,
        indent=2,
    )

print("Saved single-run artifacts in:", ART.resolve())


# ====== 5.2 K-fold Cross-Validation for Checkpoint-1 Variant ======

@torch.no_grad()
def eval_model_for_cv(m, loader):
    """
    Evaluate model on a CV fold (no need for full report, just acc + macro-F1).
    """
    m.eval()
    yT, yP = [], []
    for x, y in loader:
        x = x.to(DEVICE)
        logits = m(x)
        yP.extend(torch.argmax(logits, 1).cpu().numpy())
        yT.extend(y.numpy())
    yT = np.array(yT); yP = np.array(yP)
    acc = accuracy_score(yT, yP)
    f1m = f1_score(yT, yP, average="macro")
    return acc, f1m

def train_one_fold(fold_train_df, fold_val_df, num_epochs=5):
    """
    Train a fresh ResNet18 model on one CV fold and return validation metrics.
    """
    fold_train_loader = DataLoader(
        FrameDS(fold_train_df, True),
        batch_size=32, shuffle=True, num_workers=0, pin_memory=False
    )
    fold_val_loader = DataLoader(
        FrameDS(fold_val_df, False),
        batch_size=64, shuffle=False, num_workers=0, pin_memory=False
    )

    # New model per fold
    m = make_model(num_classes=3, pretrained=True).to(DEVICE)

    # Class weights based on THIS fold's training data
    tc_fold = fold_train_df["label"].value_counts().reindex(CLASSES, fill_value=0).astype(float)
    N_fold = tc_fold.sum(); K = len(CLASSES)
    w_fold = torch.tensor([N_fold / (K * tc_fold[c]) for c in CLASSES], dtype=torch.float32, device=DEVICE)

    crit_fold = nn.CrossEntropyLoss(weight=w_fold)
    opt_fold  = torch.optim.Adam(m.parameters(), lr=1e-4)

    for ep in range(1, num_epochs + 1):
        m.train()
        running = 0.0
        for x, y in fold_train_loader:
            x = x.to(DEVICE); y = y.to(DEVICE)
            opt_fold.zero_grad()
            logits = m(x)
            loss = crit_fold(logits, y)
            loss.backward()
            opt_fold.step()
            running += loss.item() * x.size(0)
        tr_loss = running / len(fold_train_loader.dataset)

        val_acc, val_f1 = eval_model_for_cv(m, fold_val_loader)
        print(f"  Epoch {ep:02d} | train_loss={tr_loss:.4f} | val_acc={val_acc:.3f} | val_F1={val_f1:.3f}")

    # Final metrics on validation fold
    val_acc, val_f1 = eval_model_for_cv(m, fold_val_loader)
    return val_acc, val_f1

# ---- Run Stratified K-fold CV on training data ----
K_FOLDS = 3   # can change to 5 if you have time
labels_int = train_df["label"].map(LBL).values
skf = StratifiedKFold(n_splits=K_FOLDS, shuffle=True, random_state=SEED)

cv_accs, cv_f1s = [], []

print(f"\n===== {K_FOLDS}-fold Cross-Validation for Checkpoint-1 CNN Variant =====")
for fold, (train_idx, val_idx) in enumerate(skf.split(train_df["path"], labels_int), 1):
    print(f"\n--- Fold {fold}/{K_FOLDS} ---")
    fold_train_df = train_df.iloc[train_idx].reset_index(drop=True)
    fold_val_df   = train_df.iloc[val_idx].reset_index(drop=True)

    fold_acc, fold_f1 = train_one_fold(fold_train_df, fold_val_df, num_epochs=5)
    print(f"Fold {fold} final val_acc={fold_acc:.3f}, val_macroF1={fold_f1:.3f}")

    cv_accs.append(fold_acc)
    cv_f1s.append(fold_f1)

cv_acc_mean  = float(np.mean(cv_accs))
cv_acc_std   = float(np.std(cv_accs))
cv_f1_mean   = float(np.mean(cv_f1s))
cv_f1_std    = float(np.std(cv_f1s))

print(f"\nCV accuracy (mean ± std): {cv_acc_mean:.3f} ± {cv_acc_std:.3f}")
print(f"CV macro-F1 (mean ± std): {cv_f1_mean:.3f} ± {cv_f1_std:.3f}")

with open(ART / "cv_metrics_checkpoint1.json", "w") as f:
    json.dump(
        {
            "k_folds": K_FOLDS,
            "cv_acc_mean": cv_acc_mean,
            "cv_acc_std": cv_acc_std,
            "cv_macroF1_mean": cv_f1_mean,
            "cv_macroF1_std": cv_f1_std,
            "fold_accs": [float(x) for x in cv_accs],
            "fold_macroF1s": [float(x) for x in cv_f1s],
        },
        f,
        indent=2,
    )

# ====== 5.2.1 CV Plots (for slides) ======
fold_ids = np.arange(1, K_FOLDS + 1)

# (1) CV accuracy by fold + mean + baseline
plt.figure(figsize=(5.6, 4.6))
plt.bar(fold_ids, cv_accs, width=0.6)
plt.axhline(cv_acc_mean, linestyle="--", linewidth=1, label=f"Mean CV acc = {cv_acc_mean:.3f}")
plt.axhline(baseline_acc, linestyle=":", linewidth=1, label=f"Baseline = {baseline_acc:.3f}")

plt.xticks(fold_ids, [f"Fold {i}" for i in fold_ids])
plt.ylim(0, 1)
plt.ylabel("Accuracy")
plt.title(f"{K_FOLDS}-Fold CV Accuracy (Checkpoint-1 CNN)")
plt.legend()
plt.tight_layout()
plt.savefig(ART / "cv_accuracy_by_fold_checkpoint1.png", dpi=150)
plt.close()

# (2) CV macro-F1 by fold + mean
plt.figure(figsize=(5.6, 4.6))
plt.bar(fold_ids, cv_f1s, width=0.6)
plt.axhline(cv_f1_mean, linestyle="--", linewidth=1, label=f"Mean CV F1 = {cv_f1_mean:.3f}")

plt.xticks(fold_ids, [f"Fold {i}" for i in fold_ids])
plt.ylim(0, 1)
plt.ylabel("Macro-F1")
plt.title(f"{K_FOLDS}-Fold CV Macro-F1 (Checkpoint-1 CNN)")
plt.legend()
plt.tight_layout()
plt.savefig(ART / "cv_macroF1_by_fold_checkpoint1.png", dpi=150)
plt.close()

print("Saved CV artifacts in:", ART.resolve())
# ====== End of Checkpoint-1 model + CV block ======


===== Train/Test run for Checkpoint-1 model variant =====
Epoch 01 | train_loss=1.0672 | test_acc=0.524 | macroF1=0.512
Epoch 02 | train_loss=0.9649 | test_acc=0.637 | macroF1=0.612
Epoch 03 | train_loss=0.8477 | test_acc=0.661 | macroF1=0.644
Epoch 04 | train_loss=0.7517 | test_acc=0.685 | macroF1=0.676
Epoch 05 | train_loss=0.6521 | test_acc=0.665 | macroF1=0.663
Epoch 06 | train_loss=0.5755 | test_acc=0.677 | macroF1=0.676
Epoch 07 | train_loss=0.4950 | test_acc=0.677 | macroF1=0.680
Epoch 08 | train_loss=0.4260 | test_acc=0.702 | macroF1=0.702

FINAL — Test Acc: 0.702 | Macro-F1: 0.702
Saved single-run artifacts in: /Users/wangzhuoran/Desktop/COGS 109 project/artifacts

===== 3-fold Cross-Validation for Checkpoint-1 CNN Variant =====

--- Fold 1/3 ---
  Epoch 01 | train_loss=1.0850 | val_acc=0.495 | val_F1=0.494
  Epoch 02 | train_loss=1.0119 | val_acc=0.564 | val_F1=0.543
  Epoch 03 | train_loss=0.9375 | val_acc=0.627 | val_F1=0.610
  Epoch 04 | train_loss=0.8507 | val_acc=0.649 |

In [11]:
# ---- AIC/BIC for Variant 1 ----
aic_v1, bic_v1 = compute_aic_bic(
    model,
    test_loader,
    variant_name="Variant 1 — ResNet18 FT",
    out_json_path=ART / "aic_bic_variant1.json"
)



===== Variant 1 — ResNet18 FT: AIC / BIC =====
Samples (n): 248
Parameters (k): 11178051
Negative log-likelihood (NLL): 166.03
AIC: 22356434.05
BIC: 61629719.76


### 2. Regularized CNN

In [ ]:
from sklearn.model_selection import StratifiedKFold
import numpy as np
import json

print("\n===== Variant 2 — Regularized ResNet18: Test + CV + AIC/BIC =====")

# ---------- 1) Dataloaders for Variant 2 (no strong aug) ----------
v2_train_loader = DataLoader(
    FrameDS(train_df, train=True),   # if your FrameDS ignores `train` or doesn't augment, it's fine
    batch_size=32, shuffle=True, num_workers=0, pin_memory=False
)
v2_test_loader = DataLoader(
    FrameDS(test_df, train=False),
    batch_size=64, shuffle=False, num_workers=0, pin_memory=False
)

# ---------- 2) Model + loss (same settings as your V2 training) ----------
def make_v2_model(num_classes=3, pretrained=True):
    return make_model(num_classes, pretrained)  # same resnet18

# reload best weights for test / AIC/BIC
model_v2 = make_v2_model(3, True).to(DEVICE)
model_v2.load_state_dict(torch.load(ART/"best_v2.pt", map_location=DEVICE))

# class weights
tc_v2 = train_df["label"].value_counts().reindex(CLASSES, fill_value=0).astype(float)
N_v2 = tc_v2.sum(); K = len(CLASSES)
w_v2 = torch.tensor([N_v2/(K*tc_v2[c]) for c in CLASSES], dtype=torch.float32, device=DEVICE)

criterion_v2 = nn.CrossEntropyLoss(weight=w_v2, label_smoothing=0.05)
opt_v2 = torch.optim.Adam(model_v2.parameters(), lr=5e-5, weight_decay=1e-4)

@torch.no_grad()
def eval_v2(loader, m=None):
    if m is None:
        m = model_v2
    m.eval()
    yT, yP = [], []
    for x,y in loader:
        x = x.to(DEVICE)
        logits = m(x)
        yP.extend(torch.argmax(logits,1).cpu().numpy())
        yT.extend(y.numpy())
    yT = np.array(yT); yP = np.array(yP)
    acc  = accuracy_score(yT,yP)
    f1m  = f1_score(yT,yP,average="macro")
    return acc, f1m

# ---------- 3) Final test metrics for Variant 2 ----------
v2_test_acc, v2_test_f1 = eval_v2(v2_test_loader, model_v2)
print(f"[V2] Test Accuracy   : {v2_test_acc:.3f}")
print(f"[V2] Test Macro-F1   : {v2_test_f1:.3f}")

# ---------- 4) K-fold Cross-Validation (same style as Variant 3) ----------
print("\n===== V2: K-fold Cross-Validation =====")

def train_one_fold_v2(fold_train_df, fold_val_df, num_epochs=5):
    fold_train_loader = DataLoader(
        FrameDS(fold_train_df, train=True),
        batch_size=32, shuffle=True, num_workers=0, pin_memory=False
    )
    fold_val_loader = DataLoader(
        FrameDS(fold_val_df, train=False),
        batch_size=64, shuffle=False, num_workers=0, pin_memory=False
    )

    m = make_v2_model(3, True).to(DEVICE)

    # class weights per fold
    tc_fold = fold_train_df["label"].value_counts().reindex(CLASSES, fill_value=0).astype(float)
    N_fold = tc_fold.sum(); K = len(CLASSES)
    w_fold = torch.tensor([N_fold/(K*tc_fold[c]) for c in CLASSES], dtype=torch.float32, device=DEVICE)

    crit_fold = nn.CrossEntropyLoss(weight=w_fold, label_smoothing=0.05)
    opt_fold  = torch.optim.Adam(m.parameters(), lr=5e-5, weight_decay=1e-4)

    for ep in range(1, num_epochs+1):
        m.train(); total = 0.0
        for x,y in fold_train_loader:
            x = x.to(DEVICE); y = y.to(DEVICE)
            opt_fold.zero_grad()
            logits = m(x)
            loss = crit_fold(logits,y)
            loss.backward(); opt_fold.step()
            total += loss.item()*x.size(0)
        tr_loss = total/len(fold_train_loader.dataset)
        val_acc, val_f1 = eval_v2(fold_val_loader, m)
        print(f"  Epoch {ep:02d} | train_loss={tr_loss:.4f} | val_acc={val_acc:.3f} | val_F1={val_f1:.3f}")

    val_acc, val_f1 = eval_v2(fold_val_loader, m)
    return val_acc, val_f1

K_FOLDS_V2 = 3
labels_int_v2 = train_df["label"].map(LBL).values
skf_v2 = StratifiedKFold(n_splits=K_FOLDS_V2, shuffle=True, random_state=SEED)

cv_accs_v2, cv_f1s_v2 = [], []

for fold, (tr_idx, val_idx) in enumerate(skf_v2.split(train_df["path"], labels_int_v2), 1):
    print(f"\n--- Variant 2: Fold {fold}/{K_FOLDS_V2} ---")
    fold_train_df = train_df.iloc[tr_idx].reset_index(drop=True)
    fold_val_df   = train_df.iloc[val_idx].reset_index(drop=True)
    fold_acc, fold_f1 = train_one_fold_v2(fold_train_df, fold_val_df, num_epochs=5)
    print(f"Fold {fold} final val_acc={fold_acc:.3f}, val_macroF1={fold_f1:.3f}")
    cv_accs_v2.append(fold_acc); cv_f1s_v2.append(fold_f1)

cv_acc_mean_v2 = float(np.mean(cv_accs_v2))
cv_acc_std_v2  = float(np.std(cv_accs_v2))
cv_f1_mean_v2  = float(np.mean(cv_f1s_v2))
cv_f1_std_v2   = float(np.std(cv_f1s_v2))

print(f"\n[V2] CV Accuracy (mean ± std): {cv_acc_mean_v2:.3f} ± {cv_acc_std_v2:.3f}")
print(f"[V2] CV Macro-F1 (mean ± std): {cv_f1_mean_v2:.3f} ± {cv_f1_std_v2:.3f}")

with open(ART/"cv_metrics_v2.json","w") as f:
    json.dump(
        {
            "k_folds": K_FOLDS_V2,
            "cv_acc_mean": cv_acc_mean_v2,
            "cv_acc_std": cv_acc_std_v2,
            "cv_macroF1_mean": cv_f1_mean_v2,
            "cv_macroF1_std": cv_f1_std_v2,
            "fold_accs": [float(x) for x in cv_accs_v2],
            "fold_macroF1s": [float(x) for x in cv_f1s_v2],
        },
        f,
        indent=2,
    )

# ---------- 5) AIC / BIC on test set ----------
aic_v2, bic_v2 = compute_aic_bic(
    model_v2,
    v2_test_loader,
    variant_name="Variant 2",
    out_json_path=ART/"aic_bic_v2.json"
)

# ---------- 6) Final one-line summary for slide ----------
print("\n===== Variant 2 Summary =====")
print(f"Test Accuracy      : {v2_test_acc:.3f}")
print(f"Test Macro-F1      : {v2_test_f1:.3f}")
print(f"Mean CV Accuracy   : {cv_acc_mean_v2:.3f}")
print(f"Mean CV Macro-F1   : {cv_f1_mean_v2:.3f}")
print(f"AIC                : {aic_v2:.2f}")
print(f"BIC                : {bic_v2:.2f}")
print("=====================================")



===== Variant 2 — Regularized ResNet18: Test + CV + AIC/BIC =====
[V2] Test Accuracy   : 0.694
[V2] Test Macro-F1   : 0.689

===== V2: K-fold Cross-Validation =====

--- Variant 2: Fold 1/3 ---
  Epoch 01 | train_loss=1.0987 | val_acc=0.375 | val_F1=0.361
  Epoch 02 | train_loss=1.0692 | val_acc=0.445 | val_F1=0.441
  Epoch 03 | train_loss=1.0411 | val_acc=0.503 | val_F1=0.499
  Epoch 04 | train_loss=1.0143 | val_acc=0.553 | val_F1=0.536
  Epoch 05 | train_loss=0.9805 | val_acc=0.575 | val_F1=0.566
Fold 1 final val_acc=0.575, val_macroF1=0.566

--- Variant 2: Fold 2/3 ---
  Epoch 01 | train_loss=1.0962 | val_acc=0.410 | val_F1=0.396
  Epoch 02 | train_loss=1.0653 | val_acc=0.482 | val_F1=0.487
  Epoch 03 | train_loss=1.0366 | val_acc=0.525 | val_F1=0.529
  Epoch 04 | train_loss=1.0099 | val_acc=0.570 | val_F1=0.559
  Epoch 05 | train_loss=0.9800 | val_acc=0.584 | val_F1=0.570
Fold 2 final val_acc=0.584, val_macroF1=0.570

--- Variant 2: Fold 3/3 ---
  Epoch 01 | train_loss=1.0883 | va

### 3. Augmented ResNet18

In [ ]:
# ============================================================
# Variant 3 — Augmented ResNet18 (stronger data augmentation)
# ============================================================
import random
from PIL import ImageEnhance
from sklearn.model_selection import StratifiedKFold
from matplotlib import pyplot as plt

# ---------- 1) Augmentation utilities ----------

def augment_to_tensor(img: Image.Image):
    """
    Stronger augmentation for training:
      - Convert to RGB
      - Random rotation in [-20°, +20°]
      - Random crop to 224x224 (with resize if needed)
      - Mild color jitter (brightness, contrast, saturation ∈ [0.8, 1.2])
      - Normalize with ImageNet mean/std and convert to CHW tensor
    """
    img = img.convert("RGB")

    # Random rotation
    angle = random.uniform(-20.0, 20.0)
    img = img.rotate(angle, resample=Image.BILINEAR, expand=False)

    # Random resize / crop to IMSIZE x IMSIZE (e.g., 224)
    w, h = img.size
    if w < IMSIZE or h < IMSIZE:
        # scale up so both sides >= IMSIZE
        scale = max(IMSIZE / w, IMSIZE / h)
        new_w, new_h = int(w * scale), int(h * scale)
        img = img.resize((new_w, new_h), Image.BILINEAR)
        w, h = img.size

    # If still larger than IMSIZE, random crop
    if w > IMSIZE and h > IMSIZE:
        left = random.randint(0, w - IMSIZE)
        top = random.randint(0, h - IMSIZE)
        img = img.crop((left, top, left + IMSIZE, top + IMSIZE))
    else:
        # fallback to center-ish resize
        img = img.resize((IMSIZE, IMSIZE), Image.BILINEAR)

    # Mild color jitter: brightness, contrast, saturation
    for Enhancer in (ImageEnhance.Brightness, ImageEnhance.Contrast, ImageEnhance.Color):
        factor = random.uniform(0.8, 1.2)
        img = Enhancer(img).enhance(factor)

    # To tensor + normalize (same as your original to_tensor)
    arr = np.asarray(img).astype("float32") / 255.0
    arr = (arr - IMNET_MEAN) / IMNET_STD
    arr = arr.transpose(2, 0, 1)  # CHW
    return torch.from_numpy(arr)


class AugFrameDS(Dataset):
    """
    Dataset for Augmented Variant:
      - train=True  -> strong augmentation (augment_to_tensor)
      - train=False -> plain normalization (to_tensor)
    """
    def __init__(self, frame: pd.DataFrame, train: bool = False):
        self.df = frame.reset_index(drop=True).copy()
        self.train = train

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx: int):
        row = self.df.iloc[idx]
        img = Image.open(row["path"])
        if self.train:
            x = augment_to_tensor(img)
        else:
            x = to_tensor(img)   # use your original normalization
        y = LBL[row["label"]]
        return x, y


# ---------- 2) Model factory (same as Variant 1) ----------

def make_aug_model(num_classes=3, pretrained=True):
    """
    Same architecture as Variant 1 (ResNet18),
    but we will treat this as Variant 3 with data augmentation.
    """
    return timm.create_model("resnet18.a1_in1k", pretrained=pretrained, num_classes=num_classes)


# ============================================================
# 2.1 Single Train/Test run for Augmented Variant (no CV)
# ============================================================

print("\n===== Variant 3 — Augmented ResNet18: Train/Test run =====")

# Dataloaders for this variant
aug_train_loader = DataLoader(
    AugFrameDS(train_df, train=True),
    batch_size=32, shuffle=True, num_workers=0, pin_memory=False
)
aug_test_loader = DataLoader(
    AugFrameDS(test_df, train=False),
    batch_size=64, shuffle=False, num_workers=0, pin_memory=False
)

AUG_WEIGHTS = ART / "best_aug_variant.pt"
EPOCHS_AUG = 8

# Class weights based on training data
tc_aug = train_df["label"].value_counts().reindex(CLASSES, fill_value=0).astype(float)
N_aug = tc_aug.sum()
K = len(CLASSES)
w_aug = torch.tensor([N_aug / (K * tc_aug[c]) for c in CLASSES], dtype=torch.float32, device=DEVICE)

# Define model, loss, optimizer
aug_model = make_aug_model(num_classes=3, pretrained=True).to(DEVICE)
aug_criterion = nn.CrossEntropyLoss(weight=w_aug)
aug_opt = torch.optim.Adam(aug_model.parameters(), lr=1e-4)

def train_one_epoch_aug():
    aug_model.train()
    total = 0.0
    for x, y in aug_train_loader:
        x = x.to(DEVICE); y = y.to(DEVICE)
        aug_opt.zero_grad()
        logits = aug_model(x)
        loss = aug_criterion(logits, y)
        loss.backward()
        aug_opt.step()
        total += loss.item() * x.size(0)
    return total / len(aug_train_loader.dataset)

@torch.no_grad()
def evaluate_aug(loader):
    aug_model.eval()
    yT, yP, yPR = [], [], []
    for x, y in loader:
        x = x.to(DEVICE)
        logits = aug_model(x)
        yP.extend(torch.argmax(logits, 1).cpu().numpy())
        yPR.extend(torch.softmax(logits, 1).cpu().numpy())
        yT.extend(y.numpy())
    yT = np.array(yT); yP = np.array(yP); yPR = np.array(yPR)
    acc = accuracy_score(yT, yP)
    f1m = f1_score(yT, yP, average="macro")
    rep = classification_report(yT, yP, target_names=CLASSES, output_dict=True)
    return acc, f1m, rep, yT, yP, yPR

if AUG_WEIGHTS.exists():
    print(f"Found existing weights at {AUG_WEIGHTS}, skipping training.")
    aug_model.load_state_dict(torch.load(AUG_WEIGHTS, map_location=DEVICE))
else:
    print("No existing weights found, start training Augmented Variant...")
    best_aug = 0.0
    for ep in range(1, EPOCHS_AUG + 1):
        tr_loss = train_one_epoch_aug()
        acc_ep, f1_ep, _, _, _, _ = evaluate_aug(aug_test_loader)
        print(f"Epoch {ep:02d} | train_loss={tr_loss:.4f} | test_acc={acc_ep:.3f} | macroF1={f1_ep:.3f}")
        if acc_ep > best_aug:
            best_aug = acc_ep
            torch.save(aug_model.state_dict(), AUG_WEIGHTS)
            print(f"  -> New best test acc, model saved to {AUG_WEIGHTS}")

    aug_model.load_state_dict(torch.load(AUG_WEIGHTS, map_location=DEVICE))

# Final test evaluation
aug_acc, aug_f1, aug_rep, aug_y_true, aug_y_pred, aug_y_prob = evaluate_aug(aug_test_loader)
pd.DataFrame(aug_rep).to_csv(ART / "classification_report_aug_variant_test.csv")
print(f"\n[Augmented Variant] FINAL — Test Acc: {aug_acc:.3f} | Macro-F1: {aug_f1:.3f}")

# ---------- 2.1.1 Figures for Augmented Variant (new figures) ----------

# (1) Baseline vs Augmented model accuracy on test set
plt.figure(figsize=(5, 4))
vals = [baseline_acc, aug_acc]
plt.bar(["Naive baseline", "Augmented CNN"], vals)
plt.ylim(0, 1); plt.ylabel("Accuracy")
for i, v in enumerate(vals):
    plt.text(i, v + 0.02, f"{v*100:.1f}%", ha="center")
plt.title("Accuracy: Baseline vs Augmented CNN (Test Set)")
plt.tight_layout()
plt.savefig(ART / "bar_accuracy_aug_variant.png", dpi=150)
plt.close()

# (2) Confusion matrix (Augmented variant)
cm_aug = confusion_matrix(aug_y_true, aug_y_pred, labels=[0, 1, 2])
fig, ax = plt.subplots(figsize=(5.2, 4.6))
im = ax.imshow(cm_aug, cmap="Blues")
ax.set_xticks([0, 1, 2]); ax.set_xticklabels([c.capitalize() for c in CLASSES], rotation=15)
ax.set_yticks([0, 1, 2]); ax.set_yticklabels([c.capitalize() for c in CLASSES])
for (i, j), v in np.ndenumerate(cm_aug):
    ax.text(j, i, str(v), ha="center", va="center")
ax.set_xlabel("Predicted"); ax.set_ylabel("True"); ax.set_title("Confusion Matrix (Augmented CNN)")
fig.colorbar(im, fraction=0.046, pad=0.04)
plt.tight_layout()
plt.savefig(ART / "confusion_matrix_aug_variant.png", dpi=150)
plt.close()

# (3) ROC (OvR) + macro AUC for Augmented variant
aug_y_true_ovr = label_binarize(aug_y_true, classes=[0, 1, 2])
aug_auc_macro = roc_auc_score(aug_y_true_ovr, aug_y_prob, average="macro", multi_class="ovr")
plt.figure(figsize=(5.6, 4.6))
for i, c in enumerate(CLASSES):
    fpr, tpr, _ = roc_curve(aug_y_true_ovr[:, i], aug_y_prob[:, i])
    plt.plot(fpr, tpr, label=c.capitalize())
plt.plot([0, 1], [0, 1], "--", linewidth=1)
plt.xlabel("FPR"); plt.ylabel("TPR")
plt.title(f"Augmented CNN ROC (OvR) • Macro-AUC={aug_auc_macro:.3f}")
plt.legend()
plt.tight_layout()
plt.savefig(ART / "roc_ovr_aug_variant.png", dpi=150)
plt.close()

with open(ART / "metrics_aug_variant_test.json", "w") as f:
    json.dump(
        {
            "baseline_acc": float(baseline_acc),
            "test_acc": float(aug_acc),
            "macro_f1": float(aug_f1),
            "macro_auc_ovr": float(aug_auc_macro),
        },
        f,
        indent=2,
    )

print("Saved Augmented Variant test artifacts in:", ART.resolve())


# ============================================================
# 2.2 K-fold Cross-Validation for Augmented Variant
# ============================================================

print(f"\n===== K-fold Cross-Validation for Augmented Variant =====")

@torch.no_grad()
def eval_model_for_cv_aug(m, loader):
    m.eval()
    yT, yP = [], []
    for x, y in loader:
        x = x.to(DEVICE)
        logits = m(x)
        yP.extend(torch.argmax(logits, 1).cpu().numpy())
        yT.extend(y.numpy())
    yT = np.array(yT); yP = np.array(yP)
    acc = accuracy_score(yT, yP)
    f1m = f1_score(yT, yP, average="macro")
    return acc, f1m

def train_one_fold_aug(fold_train_df, fold_val_df, num_epochs=5):
    """
    Train Augmented ResNet18 on one CV fold:
      - train fold: with augmentation
      - val fold: only normalization
    """
    fold_train_loader = DataLoader(
        AugFrameDS(fold_train_df, train=True),
        batch_size=32, shuffle=True, num_workers=0, pin_memory=False
    )
    fold_val_loader = DataLoader(
        AugFrameDS(fold_val_df, train=False),
        batch_size=64, shuffle=False, num_workers=0, pin_memory=False
    )

    m = make_aug_model(num_classes=3, pretrained=True).to(DEVICE)

    # Class weights from this fold's train data
    tc_fold = fold_train_df["label"].value_counts().reindex(CLASSES, fill_value=0).astype(float)
    N_fold = tc_fold.sum()
    K = len(CLASSES)
    w_fold = torch.tensor([N_fold / (K * tc_fold[c]) for c in CLASSES], dtype=torch.float32, device=DEVICE)
    crit_fold = nn.CrossEntropyLoss(weight=w_fold)
    opt_fold = torch.optim.Adam(m.parameters(), lr=1e-4)

    for ep in range(1, num_epochs + 1):
        m.train()
        total = 0.0
        for x, y in fold_train_loader:
            x = x.to(DEVICE); y = y.to(DEVICE)
            opt_fold.zero_grad()
            logits = m(x)
            loss = crit_fold(logits, y)
            loss.backward()
            opt_fold.step()
            total += loss.item() * x.size(0)
        tr_loss = total / len(fold_train_loader.dataset)

        val_acc, val_f1 = eval_model_for_cv_aug(m, fold_val_loader)
        print(f"  Epoch {ep:02d} | train_loss={tr_loss:.4f} | val_acc={val_acc:.3f} | val_F1={val_f1:.3f}")

    # final val metrics
    val_acc, val_f1 = eval_model_for_cv_aug(m, fold_val_loader)
    return val_acc, val_f1

K_FOLDS_AUG = 3  # same k as Variant 1 for fair comparison
labels_int_aug = train_df["label"].map(LBL).values
skf_aug = StratifiedKFold(n_splits=K_FOLDS_AUG, shuffle=True, random_state=SEED)

cv_accs_aug, cv_f1s_aug = [], []

for fold, (tr_idx, val_idx) in enumerate(skf_aug.split(train_df["path"], labels_int_aug), 1):
    print(f"\n--- Augmented Variant: Fold {fold}/{K_FOLDS_AUG} ---")
    fold_train_df = train_df.iloc[tr_idx].reset_index(drop=True)
    fold_val_df   = train_df.iloc[val_idx].reset_index(drop=True)

    fold_acc, fold_f1 = train_one_fold_aug(fold_train_df, fold_val_df, num_epochs=5)
    print(f"Fold {fold} final val_acc={fold_acc:.3f}, val_macroF1={fold_f1:.3f}")

    cv_accs_aug.append(fold_acc)
    cv_f1s_aug.append(fold_f1)

cv_acc_mean_aug = float(np.mean(cv_accs_aug))
cv_acc_std_aug  = float(np.std(cv_accs_aug))
cv_f1_mean_aug  = float(np.mean(cv_f1s_aug))
cv_f1_std_aug   = float(np.std(cv_f1s_aug))

print(f"\n[Augmented Variant] CV accuracy (mean ± std): {cv_acc_mean_aug:.3f} ± {cv_acc_std_aug:.3f}")
print(f"[Augmented Variant] CV macro-F1 (mean ± std): {cv_f1_mean_aug:.3f} ± {cv_f1_std_aug:.3f}")

with open(ART / "cv_metrics_aug_variant.json", "w") as f:
    json.dump(
        {
            "k_folds": K_FOLDS_AUG,
            "cv_acc_mean": cv_acc_mean_aug,
            "cv_acc_std": cv_acc_std_aug,
            "cv_macroF1_mean": cv_f1_mean_aug,
            "cv_macroF1_std": cv_f1_std_aug,
            "fold_accs": [float(x) for x in cv_accs_aug],
            "fold_macroF1s": [float(x) for x in cv_f1s_aug],
        },
        f,
        indent=2,
    )

# ---------- 2.2.1 CV plots for Augmented Variant ----------

fold_ids_aug = np.arange(1, K_FOLDS_AUG + 1)

# (1) CV accuracy by fold + mean + baseline
plt.figure(figsize=(5.6, 4.6))
plt.bar(fold_ids_aug, cv_accs_aug, width=0.6)
plt.axhline(cv_acc_mean_aug, linestyle="--", linewidth=1, label=f"Mean CV acc = {cv_acc_mean_aug:.3f}")
plt.axhline(baseline_acc, linestyle=":", linewidth=1, label=f"Baseline = {baseline_acc:.3f}")
plt.xticks(fold_ids_aug, [f"Fold {i}" for i in fold_ids_aug])
plt.ylim(0, 1)
plt.ylabel("Accuracy")
plt.title(f"{K_FOLDS_AUG}-Fold CV Accuracy (Augmented CNN)")
plt.legend()
plt.tight_layout()
plt.savefig(ART / "cv_accuracy_by_fold_aug_variant.png", dpi=150)
plt.close()

# (2) CV macro-F1 by fold + mean
plt.figure(figsize=(5.6, 4.6))
plt.bar(fold_ids_aug, cv_f1s_aug, width=0.6)
plt.axhline(cv_f1_mean_aug, linestyle="--", linewidth=1, label=f"Mean CV F1 = {cv_f1_mean_aug:.3f}")
plt.xticks(fold_ids_aug, [f"Fold {i}" for i in fold_ids_aug])
plt.ylim(0, 1)
plt.ylabel("Macro-F1")
plt.title(f"{K_FOLDS_AUG}-Fold CV Macro-F1 (Augmented CNN)")
plt.legend()
plt.tight_layout()
plt.savefig(ART / "cv_macroF1_by_fold_aug_variant.png", dpi=150)
plt.close()

print("Saved Augmented Variant CV artifacts in:", ART.resolve())
# =================== End of Variant 3 ===================



===== Variant 3 — Augmented ResNet18: Train/Test run =====
No existing weights found, start training Augmented Variant...
Epoch 01 | train_loss=1.0797 | test_acc=0.516 | macroF1=0.511
  -> New best test acc, model saved to artifacts/best_aug_variant.pt
Epoch 02 | train_loss=1.0064 | test_acc=0.645 | macroF1=0.627
  -> New best test acc, model saved to artifacts/best_aug_variant.pt
Epoch 03 | train_loss=0.9133 | test_acc=0.685 | macroF1=0.654
  -> New best test acc, model saved to artifacts/best_aug_variant.pt
Epoch 04 | train_loss=0.8139 | test_acc=0.706 | macroF1=0.690
  -> New best test acc, model saved to artifacts/best_aug_variant.pt
Epoch 05 | train_loss=0.7510 | test_acc=0.702 | macroF1=0.684
Epoch 06 | train_loss=0.6961 | test_acc=0.714 | macroF1=0.707
  -> New best test acc, model saved to artifacts/best_aug_variant.pt
Epoch 07 | train_loss=0.6541 | test_acc=0.710 | macroF1=0.702
Epoch 08 | train_loss=0.6197 | test_acc=0.726 | macroF1=0.712
  -> New best test acc, model saved 

In [13]:
# ---- AIC/BIC for Variant 3 (Augmented) ----
aic_v3, bic_v3 = compute_aic_bic(
    aug_model,
    aug_test_loader,
    variant_name="Variant 3 — Augmented ResNet18",
    out_json_path=ART / "aic_bic_variant3_augmented.json"
)



===== Variant 3 — Augmented ResNet18: AIC / BIC =====
Samples (n): 248
Parameters (k): 11178051
Negative log-likelihood (NLL): 157.17
AIC: 22356416.35
BIC: 61629702.06
